In [1]:
from dataclasses import asdict

import pandas as pd

from juddges.settings import NSA_DATA_PATH
import polars as pl
import pandas as pd

lf = pl.scan_parquet(NSA_DATA_PATH / "pages" / "pages_chunk_0.parquet")

In [2]:
df = lf.collect()

In [3]:
# df[2]["page"]

In [4]:
page = df[2]["page"].item()

In [59]:
from tqdm import tqdm
import re
from itertools import groupby
from bs4 import BeautifulSoup, Tag


# Function to split text by <br/> tags
def extract_data_with_br_tags(value):
    return [item.strip() for item in value.split('<br/>') if item.strip()]

# Function to preserve paragraphs with \n\n
def extract_text_preserve_paragraphs(html_element):
    paragraphs = html_element.find_all(['p', 'br'])
    text_parts = []
    for paragraph in paragraphs:
        text = paragraph.get_text(strip=True)
        if text:
            text_parts.append(text)
    return '\n\n'.join(text_parts)

def extract_przepisy(value):
    chunks = []
    x = 0
    chunks.append([])   # create an empty chunk to which we'd append in the loop
    for i in value.contents:
        if ('<br/>' not in str(i)):
            if i.get_text(strip=True) != '':
                chunks[x].append(i)
        else:
            x += 1
            chunks.append([])
    
    chunks = [chunk for chunk in chunks if len(chunk) > 0]
    przepisy = []
    
    current = {}
    for chunk in chunks:
        if isinstance(chunk[0], Tag) and "class" in chunk[0].attrs and chunk[0].attrs["class"][0] == "nakt":
            if "ustawa" in current:
                przepisy.append(current)
                current = {}
            current["ustawa"] = chunk[0].get_text(strip=True)
        else:
            if "dziennik_ustaw" in current:
                przepisy.append(current)
                current = {}
            current["dziennik_ustaw"] = chunk[0].get_text(strip=True)
            current["art"] = chunk[1].get_text(strip=True) if len(chunk) > 1 else None
            current["link"] = chunk[0]["href"] if isinstance(chunk[0], Tag) else None
    if current:
        przepisy.append(current)
    return przepisy

def extract_data(page):
    # Initialize BeautifulSoup with the HTML content
    soup = BeautifulSoup(page, 'html.parser')  # 'page' contains the HTML
    
    # Dictionary to store the extracted data
    extracted_data = {}
    
    # Find all rows containing labels and values
    rows = soup.find_all('tr', class_='niezaznaczona')
    
    # Iterate through the rows and extract label-value pairs
    for row in rows:
        label = row.find('td', class_='lista-label')
        value = row.find('td', class_='info-list-value')
    
        if label and value:
            label_text = label.get_text(strip=True)
            value_text = value.decode_contents().strip()  # Extract HTML content, including <br/>
    
            if 'Powołane przepisy' in label_text:
                extracted_data["Powołane przepisy"] = extract_przepisy(value)
            elif 'Sygn. powiązane' in label_text:
                 extracted_data[label_text] = [a['href'] for a in value.find_all('a')]
                 extracted_data[label_text] = [a.get_text(strip=True) for a in value.find_all('a')]
                 extracted_data[label_text+" link"] = [a['href'] for a in value.find_all('a')]
            elif '<br/>' in value_text or label_text in ("Hasła tematyczne", "Symbol z opisem", "Sędziowie", "Treść wyniku"):
                extracted_data[label_text] = extract_data_with_br_tags(value_text)
                if label_text == "Sędziowie":
                    function = [re.findall(r'/([^/]*)/', j) for j in extracted_data[label_text]]
                    extracted_data[label_text] = [re.sub(r'/[^/]*/', '', s).strip() for s in extracted_data[label_text]]

                    function_map = {f[0]:j for f, j in zip(function, extracted_data[label_text]) if f}
                    if "przewodniczący" in function_map and "sprawozdawca" in function_map:
                        extracted_data["przewodniczący"] = function_map["przewodniczący"]
                        extracted_data["sprawozdawca"] = function_map["sprawozdawca"]
                    elif "przewodniczący sprawozdawca" in function_map:
                        extracted_data["przewodniczący"] = function_map["przewodniczący sprawozdawca"]
                        extracted_data["sprawozdawca"] = function_map["przewodniczący sprawozdawca"]
            elif 'Data orzeczenia' in label_text:
                date_value = value.find_all('td')[0].get_text(strip=True)
                judgement_type = value.find_all('td')[1].get_text(strip=True)
                if len(judgement_type) == 0:
                    judgement_type = None
                extracted_data['Data orzeczenia'] = date_value
                extracted_data['Rodzaj orzeczenia'] = judgement_type
            else:
                extracted_data[label_text] = value_text
    
    section_headers = soup.find_all('div', class_='lista-label')
    for header in section_headers:
        next_section = header.find_next('span', class_='info-list-value-uzasadnienie')
        if next_section:
            header_text = header.get_text(strip=True)
            extracted_data[header_text] = extract_text_preserve_paragraphs(next_section)
    
    return extracted_data

l = []

for item in df.iter_rows(named=True):
    # if item["doc_id"] == "/doc/C099C5930B":
    # if item["doc_id"] == "/doc/0B08733C97":
    # if item["doc_id"] == "/doc/E1450B7375":
    if True:
        print(item["doc_id"])
        data = extract_data(item["page"])
        data["doc_id"] = item["doc_id"]
        data["page"] = item["page"]
        l.append(data)

/doc/BC8847077B
/doc/C8D885030F
/doc/6447E5BA57
/doc/B4984CED3A
/doc/21CBD4642A
/doc/6FC64DBC65
/doc/5340390CB7
/doc/0B08733C97
/doc/815585AD33
/doc/674834E183
/doc/4818165334
/doc/D11640B606
/doc/216D11D62E
/doc/BB40BD0398
/doc/EF2AB74A63
/doc/40B6D5A47A
/doc/787EBFA9B0
/doc/352E075516
/doc/9699410DE9
/doc/4D067E5E86
/doc/8354B48D93
/doc/00919830B7
/doc/E3C59E7A46
/doc/65C8731F3F
/doc/245ED36A74
/doc/D466E47BE5
/doc/6BABCF1CCA
/doc/6089557FD6
/doc/ABA50C40F3
/doc/CA1B82803F
/doc/179A781DED
/doc/615EC1E870
/doc/718407AA76
/doc/EE3DFB5A88
/doc/866ACEC25B
/doc/55E8A1E9C0
/doc/A566AE9C98
/doc/0DCFBE5EFD
/doc/B73BB23A74
/doc/FE5AC12EB1
/doc/33DA98C95B
/doc/4CE2026572
/doc/8A0D1B7134
/doc/A5C239CC09
/doc/4731333329
/doc/721E468642
/doc/1A0CECE6BF
/doc/85E68A960A
/doc/C5614186A2
/doc/BCFF41CDCE
/doc/FF8E224CF9
/doc/4A870034BC
/doc/FA6D521061
/doc/46289986BA
/doc/7ABA997816
/doc/8CA09BCBF8
/doc/0A3D557F3F
/doc/CB132298BE
/doc/78C6ADA58A
/doc/E27EA6829C
/doc/54932EA62F
/doc/2F32FE1AF2
/doc/81B

In [60]:
l[0]["Powołane przepisy"]

[{'dziennik_ustaw': 'Dz.U. 2016 poz 718',
  'art': 'art. 220 par.3, art. 58 par.3',
  'link': 'http://isap.sejm.gov.pl/DetailsServlet?id=WDU20160000718',
  'ustawa': 'Ustawa z dnia 30 sierpnia 2002 r. - Prawo o postępowaniu przed sądami administracyjnymi - tekst jednolity'}]

In [61]:
df = pd.DataFrame(l)

In [62]:
df

,Data orzeczenia,Rodzaj orzeczenia,Data wpływu,Sąd,Sędziowie,przewodniczący,sprawozdawca,Symbol z opisem,Hasła tematyczne,Skarżony organ,...,Sentencja,Uzasadnienie,doc_id,page,Publikacja w u.z.o.,Tezy,Sygn. powiązane,Sygn. powiązane link,Zdanie odrębne,Info. o glosach
0,2016-10-26,orzeczenie prawomocne,2016-09-21,Wojewódzki Sąd Administracyjny w Olsztynie,[S. Beata Jezielska],S. Beata Jezielska,S. Beata Jezielska,[6042 Gry losowe i zakłady wzajemne],[Odrzucenie skargi],Dyrektor Izby Celnej,...,Wojewódzki Sąd Administracyjny w Olsztynie w s...,Spółka A z siedzibą w W - reprezentowana przez...,/doc/BC8847077B,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-14,orzeczenie nieprawomocne,2021-04-02,Wojewódzki Sąd Administracyjny w Poznaniu,"[Izabela Paluszyńska, Piotr Ławrynowicz, Walen...",Izabela Paluszyńska,Izabela Paluszyńska,[6550],[Administracyjne postępowanie],Dyrektor Oddziału Regionalnego Agencji Restruk...,...,Dnia 14 października 2021 roku Wojewódzki Sąd ...,Decyzją z [...] lutego 2020r. Nr [...] Kierown...,/doc/C8D885030F,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,NaN,NaN,NaN,NaN,NaN
2,2005-10-13,orzeczenie prawomocne,2005-01-12,Wojewódzki Sąd Administracyjny w Opolu,"[Grażyna Jeżewska, Jerzy Krupiński, Teresa Cisyk]",Jerzy Krupiński,Teresa Cisyk,[6192 Funkcjonariusze Policji],"[Zatrudnienie, Organizacje społeczne, Policja]",Komendant Policji,...,Wojewódzki Sąd Administracyjny w Opolu w skład...,"Komendant Miejski Policji w O., rozkazem perso...",/doc/6447E5BA57,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",ONSAiWSA 2006 4 poz. 116,Przepisy art. 32 ust. 1 i 2 ustawy z dnia 23 m...,NaN,NaN,NaN,NaN
3,2010-04-02,orzeczenie prawomocne,2008-11-12,Naczelny Sąd Administracyjny,"[Bogusław Woźniak, Grzegorz Borkowski, Stefan ...",Grzegorz Borkowski,Bogusław Woźniak,[6119 Inne o symbolu podstawowym 611],[Inne],Dyrektor Izby Skarbowej,...,Naczelny Sąd Administracyjny w składzie: Przew...,Wojewódzki Sąd Administracyjny we Wrocławiu wy...,/doc/B4984CED3A,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,NaN,[I SA/Wr 183/08 - Wyrok WSA we Wrocławiu z 200...,[/doc/0233673148],NaN,NaN
4,2015-05-19,None,2014-02-10,Wojewódzki Sąd Administracyjny w Warszawie,[Maciej Kurasz],Maciej Kurasz,Maciej Kurasz,"[6112 Podatek dochodowy od osób fizycznych, w ...",[Odrzucenie skargi kasacyjnej],Dyrektor Izby Skarbowej,...,Dnia 19 maja 2015 r. Wojewódzki Sąd Administra...,Wojewódzki Sąd Administracyjny w Warszawie wyr...,/doc/21CBD4642A,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2008-08-27,orzeczenie prawomocne,2008-04-03,Wojewódzki Sąd Administracyjny w Białymstoku,"[Elżbieta Trykoszko, Jacek Pruszyński, Urszula...",Urszula Barbara Rymarska,Urszula Barbara Rymarska,[6110 Podatek od towarów i usług],[Podatek od towarów i usług],Dyrektor Izby Skarbowej,...,Wojewódzki Sąd Administracyjny w Białymstoku w...,"Decyzją z [...] stycznia 2004 r. Nr [...], Dyr...",/doc/B7BDD5EB7D,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,NaN,NaN,NaN,NaN,NaN
49996,2006-06-20,orzeczenie prawomocne,2006-05-04,Wojewódzki Sąd Administracyjny w Łodzi,"[Barbara Rymaszewska, Ewa Markiewicz, Renata K...",Barbara Rymaszewska,Renata Kubot-Szustowska,[6153 Warunki zabudowy terenu],NaN,Samorządowe Kolegium Odwoławcze,...,Wojewódzki Sąd Administracyjny w Łodzi – II Wy...,"Zarządzeniem z dnia 15.maja 2006r., doręczonym...",/doc/675BFA18CE,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,NaN,NaN,NaN,NaN,NaN
49997,2006-06-27,orzeczenie prawomocne,2006-05-25,Wojewódzki Sąd Administracyjny w Lublinie,"[Ewa Ibrom, Jerzy Drwal, Leszek Leszczyński]",Ewa Ibrom,Ewa Ibrom,"[6019 Inne, o symbolu podstawowym 601, 638 Sp...",[Egzekucyjne postępowanie],Inspektor Nadzoru Budowlanego,...,Wojewódzki Sąd Administracyjny w Lublinie w sk...,Postanowieniem z dnia [...] r. Powiatowy Inspe...,/doc/245

In [ ]:
for i, entry in df.iterrows():
    if not isinstance(entry["Dziennik ustaw"], list):
        continue
    if not len(entry['Dziennik ustaw']) == len(entry["Dziennik ustaw link"]) == len(entry["Artykuły"]) == len(entry["Ustawa"]):
        print(entry)
        print(entry["Dziennik ustaw"])
        print(entry["Dziennik ustaw link"])
        print(entry["Artykuły"])
        print(entry["Ustawa"])
        break